In [78]:
import pandas as pd
import numpy as np
import random
from copy import deepcopy as copy

In [79]:
dice_values = [(4, 1), (6, 2), (8, 3), (10, 4), (10, 4), (12, 5), (20, 6)]
goal = 20
player_1_total = 0
player_2_total = 0

In [80]:
player_1_money = 100
player_2_money = 100

In [81]:
data = pd.DataFrame(columns=["round", "player 1 strategy", 
                             "player 2 strategy", 
                             "player 1 result", 
                             "player 2 result", 
                             "player 1 money", 
                             "player 2 money"])

In [82]:
class Player:
    def __init__(self, name, strategy, money, active):
        self.name = name
        self.strategy = strategy
        self.money = money
        self.active = active
        self.total = 0
    def __repr__(self):
        return self.name
class Die:
    def __init__(self, value, cost):
        self.value = value
        self.cost = cost
    def __repr__(self):
        return str(self.value)
original_dice_list = [Die(x, y) for x, y in dice_values]
original_dice_list

[4, 6, 8, 10, 10, 12, 20]

In [83]:
def StartHighNeverBustStrategy(total, goal, money, dice):
    dice.sort(reverse=True, key=lambda x: x.value)
    print(dice)
    gap = goal - total
    gap
    if gap<0:
        raise Exception
    for y in dice:
        y
        if y.value<=gap and y.cost<=money:
            return y
    return "Hold"

def StartHighSometimesBustStrategy(total, goal, money, dice):
    dice.sort(reverse=True, key=lambda x: x.value)
    print(dice)
    gap = goal - total
    gap
    if gap<0:
        raise Exception
    for y in dice:
        y
        if ((y.value*2)/3)<=gap and y.cost<=money:
            return y
    return "Hold"

def StartHighFrequentlyBustStrategy(total, goal, money, dice):
    dice.sort(reverse=True, key=lambda x: x.value)
    print(dice)
    gap = goal - total
    gap
    if gap<0:
        raise Exception
    for y in dice:
        y
        if (y.value/2)<=gap and y.cost<=money:
            return y
    return "Hold"

def StartLowStrategy(total, goal, money, dice):
    dice.sort(key=lambda x: x.value)
    print(dice)
    gap = goal - total
    print(gap)
    if gap<0:
        raise Exception
    for y in dice:
        print(y)
        if y.value<=gap and y.cost<=money:
            return y
    return "Hold"

def WildCardStrategy(total, goal, money, dice):
    print(dice)
    return random.choice(dice)
        

In [84]:
def RunGame(player_1, player_2, dice_list, results_df, goal = 20, blind = 1, rounds=None):
    player_1 = copy(player_1)
    player_2 = copy(player_2)
    pot = 0
    round_number = 1
    print(dice_list)
    dice_list = copy(dice_list)
    initial_dice_list = copy(dice_list)
    if random.random()<0.5:
        player_order = [player_1, player_2]
    else:
        player_order = [player_2, player_1]
    player_1.active = True
    player_2.active = True
    while (rounds is None or round_number<=rounds) and (player_1.money>(10*blind) and player_2.money>(10*blind)):
        player_1.money = player_1.money - blind
        pot = pot + blind
        player_2.money = player_2.money - blind
        pot = pot + blind
        while len(dice_list)>0 and (player_1.active is True or player_2.active is True):
            for z in player_order:
                if z.active is True:
                    try:
                        die = z.strategy(z.total, goal, z.money, dice_list)
                        print("Die is {}".format(die))
                    except Exception as e:
                        z.active = False
                        z.total = 0
                        die = "Hold"
                        print("Player busted")
                        print(e)
                    if die != "Hold":
                        z.total = z.total + (random.randint(1, die.value))
                        if z.total>goal:
                            z.active = False
                            print("{} busted".format(z.name))
                        z.money = z.money - die.cost
                        pot = pot + die.cost
                        print(pot)
                        dice_list.remove(die)
                        print("Removing {} from dice list".format(die))
                        dice_list
                    else:
                        z.active = False
        if player_1.total> goal:
            player_1.total = 0
        if player_2.total > goal:
            player_2.total = 0
        if player_1.total == player_2.total:
            player_1.money = player_1.money + (pot/2)
            player_2.money = player_2.money + (pot/2)
            pot = 0
        if player_1.total > player_2.total:
            player_1.money = player_1.money + pot
            pot = 0
        if player_2.total > player_1.total:
            player_2.money = player_2.money + pot
            pot = 0
        results_dict = {"round": str(round_number), 
                        "player 1 strategy": player_1.strategy, 
                            "player 2 strategy": player_2.strategy, 
                            "player 1 result": player_1.total, 
                            "player 2 result": player_2.total, 
                            "player 1 money": player_1.money, 
                            "player 2 money": player_2.money}
        print(results_dict)
        results_df = results_df.append(results_dict, ignore_index=True)
        round_number = round_number + 1
        print("Round Number is {}".format(round_number))
        dice_list = copy(initial_dice_list)
        print(dice_list)
        player_1.total = 0
        player_2.total = 0
        player_1.active = True
        player_2.active = True
        player_order = player_order[::-1]
    return results_df


        

In [85]:
start_high_never_bust = Player("Player 1", StartHighNeverBustStrategy, player_1_money, True)
start_low = Player("Player 2", StartLowStrategy, player_2_money, True)
start_high_sometimes_bust = Player("Player 3", StartHighSometimesBustStrategy, player_1_money, True)
start_high_frequently_bust = Player("Player 4", StartHighFrequentlyBustStrategy, player_2_money, True)
wild_card = Player("Player 5", WildCardStrategy, player_1_money, True)

In [86]:
start_low.strategy(0, 20, 100, original_dice_list)

[4, 6, 8, 10, 10, 12, 20]
20
4


4

In [87]:
results_1 = RunGame(start_high_never_bust, start_low, original_dice_list, data)
results_1

[4, 6, 8, 10, 10, 12, 20]
[4, 6, 8, 10, 10, 12, 20]
20
4
Die is 4
3
Removing 4 from dice list
[20, 12, 10, 10, 8, 6]
Die is 20
9
Removing 20 from dice list
[6, 8, 10, 10, 12]
17
6
Die is 6
11
Removing 6 from dice list
[12, 10, 10, 8]
Die is 12
16
Removing 12 from dice list
[8, 10, 10]
15
8
Die is 8
19
Removing 8 from dice list
[10, 10]
Die is 10
23
Removing 10 from dice list
[10]
10
10
Die is 10
27
Removing 10 from dice list
[]
Die is Hold
{'round': '1', 'player 1 strategy': <function StartHighNeverBustStrategy at 0x7fd680469a60>, 'player 2 strategy': <function StartLowStrategy at 0x7fd680469d08>, 'player 1 result': 10, 'player 2 result': 19, 'player 1 money': 84, 'player 2 money': 116}
Round Number is 2
[4, 6, 8, 10, 10, 12, 20]
[20, 12, 10, 10, 8, 6, 4]
Die is 20
8
Removing 20 from dice list
[4, 6, 8, 10, 10, 12]
20
4
Die is 4
9
Removing 4 from dice list
[12, 10, 10, 8, 6]
Die is 6
11
Removing 6 from dice list
[8, 10, 10, 12]
19
8
Die is 8
14
Removing 8 from dice list
[12, 10, 10]
Di

,round,player 1 strategy,player 2 strategy,player 1 result,player 2 result,player 1 money,player 2 money
0,1,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartLowStrategy at 0x7fd680469d08>,10,19,84,116
1,2,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartLowStrategy at 0x7fd680469d08>,19,13,93,107
2,3,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartLowStrategy at 0x7fd680469d08>,16,12,113,87
3,4,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartLowStrategy at 0x7fd680469d08>,17,15,130,70
4,5,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartLowStrategy at 0x7fd680469d08>,11,11,131.5,68.5
5,6,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartLowStrategy at 0x7fd680469d08>,17,15,138.5,61.5
6,7,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartLowStrategy at 0x7fd680469d08>,16,13,145.5,54.5
7,8,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartLowStrategy at 0x7fd680469d08>,17,12,152.5,47.5
8,9,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartLowStrategy at 0x7fd680469d08>,16,18,142.5,57.5
9,10,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartLowStrategy at 0x7fd680469d08>,12,11,149.5,50.5


In [88]:
results_2 = RunGame(start_high_never_bust, start_high_sometimes_bust, original_dice_list, data)
results_2

[4, 6, 8, 10, 10, 12, 20]
[20, 12, 10, 10, 8, 6, 4]
Die is 20
8
Removing 20 from dice list
[12, 10, 10, 8, 6, 4]
Die is 12
13
Removing 12 from dice list
[10, 10, 8, 6, 4]
Die is 10
17
Removing 10 from dice list
[10, 8, 6, 4]
Die is 8
20
Removing 8 from dice list
[10, 6, 4]
Die is 10
24
Removing 10 from dice list
[6, 4]
Die is 6
26
Removing 6 from dice list
[4]
Die is 4
27
Removing 4 from dice list
[]
Die is Hold
{'round': '1', 'player 1 strategy': <function StartHighNeverBustStrategy at 0x7fd680469a60>, 'player 2 strategy': <function StartHighSometimesBustStrategy at 0x7fd680469400>, 'player 1 result': 15, 'player 2 result': 10, 'player 1 money': 116, 'player 2 money': 84}
Round Number is 2
[4, 6, 8, 10, 10, 12, 20]
[20, 12, 10, 10, 8, 6, 4]
Die is 20
8
Removing 20 from dice list
[12, 10, 10, 8, 6, 4]
Die is 12
13
Removing 12 from dice list
[10, 10, 8, 6, 4]
Die is 10
17
Removing 10 from dice list
[10, 8, 6, 4]
Die is 10
21
Removing 10 from dice list
[8, 6, 4]
Die is 8
24
Removing 8 fr

,round,player 1 strategy,player 2 strategy,player 1 result,player 2 result,player 1 money,player 2 money
0,1,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighSometimesBustStrategy at 0x...,15,10,116,84
1,2,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighSometimesBustStrategy at 0x...,18,6,129,71
2,3,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighSometimesBustStrategy at 0x...,17,18,115,85
3,4,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighSometimesBustStrategy at 0x...,15,16,100,100
4,5,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighSometimesBustStrategy at 0x...,19,0,114,86
5,6,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighSometimesBustStrategy at 0x...,20,18,126,74
6,7,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighSometimesBustStrategy at 0x...,19,18,137,63
7,8,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighSometimesBustStrategy at 0x...,17,14,157,43
8,9,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighSometimesBustStrategy at 0x...,17,18,143,57
9,10,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighSometimesBustStrategy at 0x...,19,19,146,54


In [89]:
results_3 = RunGame(start_high_never_bust, start_high_frequently_bust, original_dice_list, data)
results_3

[4, 6, 8, 10, 10, 12, 20]
[20, 12, 10, 10, 8, 6, 4]
Die is 20
8
Removing 20 from dice list
[12, 10, 10, 8, 6, 4]
Die is 12
13
Removing 12 from dice list
[10, 10, 8, 6, 4]
Die is 10
17
Removing 10 from dice list
[10, 8, 6, 4]
Die is 10
21
Removing 10 from dice list
[8, 6, 4]
Die is 8
24
Removing 8 from dice list
[6, 4]
Die is Hold
[6, 4]
Die is Hold
{'round': '1', 'player 1 strategy': <function StartHighNeverBustStrategy at 0x7fd680469a60>, 'player 2 strategy': <function StartHighFrequentlyBustStrategy at 0x7fd680469488>, 'player 1 result': 18, 'player 2 result': 19, 'player 1 money': 90, 'player 2 money': 110}
Round Number is 2
[4, 6, 8, 10, 10, 12, 20]
[20, 12, 10, 10, 8, 6, 4]
Die is 20
8
Removing 20 from dice list
[12, 10, 10, 8, 6, 4]
Die is 12
13
Removing 12 from dice list
[10, 10, 8, 6, 4]
Die is 10
17
Removing 10 from dice list
[10, 8, 6, 4]
Die is 10
21
Removing 10 from dice list
[8, 6, 4]
Die is 8
24
Removing 8 from dice list
[6, 4]
Die is 6
26
Removing 6 from dice list
[4]
Di


[4, 6, 8, 10, 10, 12, 20]
[20, 12, 10, 10, 8, 6, 4]
Die is 20
8
Removing 20 from dice list
[12, 10, 10, 8, 6, 4]
Die is 12
13
Removing 12 from dice list
[10, 10, 8, 6, 4]
Die is Hold
[10, 10, 8, 6, 4]
Die is 10
17
Removing 10 from dice list
[10, 8, 6, 4]
Die is 10
21
Removing 10 from dice list
[8, 6, 4]
Die is 4
22
Removing 4 from dice list
[8, 6]
Die is Hold
{'round': '13', 'player 1 strategy': <function StartHighNeverBustStrategy at 0x7fd680469a60>, 'player 2 strategy': <function StartHighFrequentlyBustStrategy at 0x7fd680469488>, 'player 1 result': 17, 'player 2 result': 19, 'player 1 money': 158, 'player 2 money': 42}
Round Number is 14
[4, 6, 8, 10, 10, 12, 20]
[20, 12, 10, 10, 8, 6, 4]
Die is 20
8
Removing 20 from dice list
[12, 10, 10, 8, 6, 4]
Die is 12
13
Removing 12 from dice list
[10, 10, 8, 6, 4]
Die is 10
17
Removing 10 from dice list
[10, 8, 6, 4]
Die is 10
21
Removing 10 from dice list
[8, 6, 4]
Die is 8
24
Removing 8 from dice list
[6, 4]
Die is 6
26
Removing 6 from di

,round,player 1 strategy,player 2 strategy,player 1 result,player 2 result,player 1 money,player 2 money
0,1,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighFrequentlyBustStrategy at 0...,18,19,90,110
1,2,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighFrequentlyBustStrategy at 0...,20,10,102,98
2,3,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighFrequentlyBustStrategy at 0...,14,20,86,114
3,4,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighFrequentlyBustStrategy at 0...,20,19,100,100
4,5,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighFrequentlyBustStrategy at 0...,17,0,109,91
5,6,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighFrequentlyBustStrategy at 0...,18,0,122,78
6,7,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighFrequentlyBustStrategy at 0...,16,19,108,92
7,8,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighFrequentlyBustStrategy at 0...,13,0,120,80
8,9,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighFrequentlyBustStrategy at 0...,18,0,136,64
9,10,<function StartHighNeverBustStrategy at 0x7fd6...,<function StartHighFrequentlyBustStrategy at 0...,19,0,153,47


In [90]:
results_4 = RunGame(start_low, start_high_sometimes_bust, original_dice_list, data)
results_4

[4, 6, 8, 10, 10, 12, 20]
[20, 12, 10, 10, 8, 6, 4]
Die is 20
8
Removing 20 from dice list
[4, 6, 8, 10, 10, 12]
20
4
Die is 4
9
Removing 4 from dice list
[12, 10, 10, 8, 6]
Die is 12
14
Removing 12 from dice list
[6, 8, 10, 10]
17
6
Die is 6
16
Removing 6 from dice list
[10, 10, 8]
Die is 10
20
Removing 10 from dice list
[8, 10]
12
8
Die is 8
23
Removing 8 from dice list
[10]
Die is Hold
[10]
7
10
Die is Hold
{'round': '1', 'player 1 strategy': <function StartLowStrategy at 0x7fd680469d08>, 'player 2 strategy': <function StartHighSometimesBustStrategy at 0x7fd680469400>, 'player 1 result': 13, 'player 2 result': 15, 'player 1 money': 93, 'player 2 money': 107}
Round Number is 2
[4, 6, 8, 10, 10, 12, 20]
[4, 6, 8, 10, 10, 12, 20]
20
4
Die is 4
3
Removing 4 from dice list
[20, 12, 10, 10, 8, 6]
Die is 20
9
Removing 20 from dice list
[6, 8, 10, 10, 12]
19
6
Die is 6
11
Removing 6 from dice list
[12, 10, 10, 8]
Die is 12
16
Removing 12 from dice list
[8, 10, 10]
14
8
Die is 8
19
Removing 

,round,player 1 strategy,player 2 strategy,player 1 result,player 2 result,player 1 money,player 2 money
0,1,<function StartLowStrategy at 0x7fd680469d08>,<function StartHighSometimesBustStrategy at 0x...,13,15,93,107
1,2,<function StartLowStrategy at 0x7fd680469d08>,<function StartHighSometimesBustStrategy at 0x...,13,14,82,118
2,3,<function StartLowStrategy at 0x7fd680469d08>,<function StartHighSometimesBustStrategy at 0x...,13,20,71,129
3,4,<function StartLowStrategy at 0x7fd680469d08>,<function StartHighSometimesBustStrategy at 0x...,16,15,82,118
4,5,<function StartLowStrategy at 0x7fd680469d08>,<function StartHighSometimesBustStrategy at 0x...,12,16,71,129
5,6,<function StartLowStrategy at 0x7fd680469d08>,<function StartHighSometimesBustStrategy at 0x...,11,19,64,136
6,7,<function StartLowStrategy at 0x7fd680469d08>,<function StartHighSometimesBustStrategy at 0x...,14,18,55,145
7,8,<function StartLowStrategy at 0x7fd680469d08>,<function StartHighSometimesBustStrategy at 0x...,14,16,48,152
8,9,<function StartLowStrategy at 0x7fd680469d08>,<function StartHighSometimesBustStrategy at 0x...,14,19,40,160
9,10,<function StartLowStrategy at 0x7fd680469d08>,<function StartHighSometimesBustStrategy at 0x...,17,18,29,171


In [91]:
results_5 = RunGame(start_low, start_high_frequently_bust, original_dice_list, data)
results_5

[4, 6, 8, 10, 10, 12, 20]
[20, 12, 10, 10, 8, 6, 4]
Die is 20
8
Removing 20 from dice list
[4, 6, 8, 10, 10, 12]
20
4
Die is 4
9
Removing 4 from dice list
[12, 10, 10, 8, 6]
Die is 12
14
Removing 12 from dice list
[6, 8, 10, 10]
19
6
Die is 6
16
Removing 6 from dice list
[10, 10, 8]
Die is 10
20
Removing 10 from dice list
[8, 10]
18
8
Die is 8
23
Removing 8 from dice list
[10]
Die is Hold
[10]
12
10
Die is 10
27
Removing 10 from dice list
{'round': '1', 'player 1 strategy': <function StartLowStrategy at 0x7fd680469d08>, 'player 2 strategy': <function StartHighFrequentlyBustStrategy at 0x7fd680469488>, 'player 1 result': 10, 'player 2 result': 16, 'player 1 money': 89, 'player 2 money': 111}
Round Number is 2
[4, 6, 8, 10, 10, 12, 20]
[4, 6, 8, 10, 10, 12, 20]
20
4
Die is 4
3
Removing 4 from dice list
[20, 12, 10, 10, 8, 6]
Die is 20
9
Removing 20 from dice list
[6, 8, 10, 10, 12]
16
6
Die is 6
11
Removing 6 from dice list
[12, 10, 10, 8]
Die is 12
Player 4 busted
16
Removing 12 from di

,round,player 1 strategy,player 2 strategy,player 1 result,player 2 result,player 1 money,player 2 money
0,1,<function StartLowStrategy at 0x7fd680469d08>,<function StartHighFrequentlyBustStrategy at 0...,10,16,89,111
1,2,<function StartLowStrategy at 0x7fd680469d08>,<function StartHighFrequentlyBustStrategy at 0...,13,0,101,99
2,3,<function StartLowStrategy at 0x7fd680469d08>,<function StartHighFrequentlyBustStrategy at 0...,19,18,108,92
3,4,<function StartLowStrategy at 0x7fd680469d08>,<function StartHighFrequentlyBustStrategy at 0...,12,20,101,99
4,5,<function StartLowStrategy at 0x7fd680469d08>,<function StartHighFrequentlyBustStrategy at 0...,13,20,86,114
5,6,<function StartLowStrategy at 0x7fd680469d08>,<function StartHighFrequentlyBustStrategy at 0...,17,0,97,103
6,7,<function StartLowStrategy at 0x7fd680469d08>,<function StartHighFrequentlyBustStrategy at 0...,12,0,109,91
7,8,<function StartLowStrategy at 0x7fd680469d08>,<function StartHighFrequentlyBustStrategy at 0...,15,17,102,98
8,9,<function StartLowStrategy at 0x7fd680469d08>,<function StartHighFrequentlyBustStrategy at 0...,11,18,91,109
9,10,<function StartLowStrategy at 0x7fd680469d08>,<function StartHighFrequentlyBustStrategy at 0...,14,0,107,93


In [92]:
results_6 = RunGame(start_low, wild_card, original_dice_list, data)
results_6

[4, 6, 8, 10, 10, 12, 20]
[4, 6, 8, 10, 10, 12, 20]
20
4
Die is 4
3
Removing 4 from dice list
[6, 8, 10, 10, 12, 20]
Die is 10
7
Removing 10 from dice list
[6, 8, 10, 12, 20]
17
6
Die is 6
9
Removing 6 from dice list
[8, 10, 12, 20]
Die is 8
12
Removing 8 from dice list
[10, 12, 20]
14
10
Die is 10
16
Removing 10 from dice list
[12, 20]
Die is 20
Player 5 busted
22
Removing 20 from dice list
[12]
6
12
Die is Hold
{'round': '1', 'player 1 strategy': <function StartLowStrategy at 0x7fd680469d08>, 'player 2 strategy': <function WildCardStrategy at 0x7fd6804696a8>, 'player 1 result': 14, 'player 2 result': 0, 'player 1 money': 114, 'player 2 money': 86}
Round Number is 2
[4, 6, 8, 10, 10, 12, 20]
[4, 6, 8, 10, 10, 12, 20]
Die is 20
8
Removing 20 from dice list
[4, 6, 8, 10, 10, 12]
20
4
Die is 4
9
Removing 4 from dice list
[6, 8, 10, 10, 12]
Die is 10
13
Removing 10 from dice list
[6, 8, 10, 12]
18
6
Die is 6
15
Removing 6 from dice list
[8, 10, 12]
Die is 8
18
Removing 8 from dice list
[1

,round,player 1 strategy,player 2 strategy,player 1 result,player 2 result,player 1 money,player 2 money
0,1,<function StartLowStrategy at 0x7fd680469d08>,<function WildCardStrategy at 0x7fd6804696a8>,14,0,114,86
1,2,<function StartLowStrategy at 0x7fd680469d08>,<function WildCardStrategy at 0x7fd6804696a8>,14,18,106,94
2,3,<function StartLowStrategy at 0x7fd680469d08>,<function WildCardStrategy at 0x7fd6804696a8>,9,0,122,78
3,4,<function StartLowStrategy at 0x7fd680469d08>,<function WildCardStrategy at 0x7fd6804696a8>,15,14,140,60
4,5,<function StartLowStrategy at 0x7fd680469d08>,<function WildCardStrategy at 0x7fd6804696a8>,13,0,159,41
5,6,<function StartLowStrategy at 0x7fd680469d08>,<function WildCardStrategy at 0x7fd6804696a8>,13,0,179,21
6,7,<function StartLowStrategy at 0x7fd680469d08>,<function WildCardStrategy at 0x7fd6804696a8>,11,0,192,8


In [93]:
results_7 = RunGame(start_high_never_bust, wild_card, original_dice_list, data)
results_7

[4, 6, 8, 10, 10, 12, 20]
[20, 12, 10, 10, 8, 6, 4]
Die is 20
8
Removing 20 from dice list
[12, 10, 10, 8, 6, 4]
Die is 10
12
Removing 10 from dice list
[12, 10, 8, 6, 4]
Die is 12
17
Removing 12 from dice list
[10, 8, 6, 4]
Die is 10
21
Removing 10 from dice list
[8, 6, 4]
Die is 8
24
Removing 8 from dice list
[6, 4]
Die is 6
Player 5 busted
26
Removing 6 from dice list
[4]
Die is 4
27
Removing 4 from dice list
{'round': '1', 'player 1 strategy': <function StartHighNeverBustStrategy at 0x7fd680469a60>, 'player 2 strategy': <function WildCardStrategy at 0x7fd6804696a8>, 'player 1 result': 20, 'player 2 result': 0, 'player 1 money': 111, 'player 2 money': 89}
Round Number is 2
[4, 6, 8, 10, 10, 12, 20]
[4, 6, 8, 10, 10, 12, 20]
Die is 12
7
Removing 12 from dice list
[20, 10, 10, 8, 6, 4]
Die is 20
13
Removing 20 from dice list
[10, 10, 8, 6, 4]
Die is 10
17
Removing 10 from dice list
[10, 8, 6, 4]
Die is 6
19
Removing 6 from dice list
[10, 8, 4]
Die is 10
23
Removing 10 from dice list
[

,round,player 1 strategy,player 2 strategy,player 1 result,player 2 result,player 1 money,player 2 money
0,1,<function StartHighNeverBustStrategy at 0x7fd6...,<function WildCardStrategy at 0x7fd6804696a8>,20,0,111,89
1,2,<function StartHighNeverBustStrategy at 0x7fd6...,<function WildCardStrategy at 0x7fd6804696a8>,17,19,101,99
2,3,<function StartHighNeverBustStrategy at 0x7fd6...,<function WildCardStrategy at 0x7fd6804696a8>,13,0,110,90
3,4,<function StartHighNeverBustStrategy at 0x7fd6...,<function WildCardStrategy at 0x7fd6804696a8>,16,0,127,73
4,5,<function StartHighNeverBustStrategy at 0x7fd6...,<function WildCardStrategy at 0x7fd6804696a8>,18,18,133.5,66.5
5,6,<function StartHighNeverBustStrategy at 0x7fd6...,<function WildCardStrategy at 0x7fd6804696a8>,19,20,122.5,77.5
6,7,<function StartHighNeverBustStrategy at 0x7fd6...,<function WildCardStrategy at 0x7fd6804696a8>,19,19,128,72
7,8,<function StartHighNeverBustStrategy at 0x7fd6...,<function WildCardStrategy at 0x7fd6804696a8>,19,10,141,59
8,9,<function StartHighNeverBustStrategy at 0x7fd6...,<function WildCardStrategy at 0x7fd6804696a8>,11,0,152,48
9,10,<function StartHighNeverBustStrategy at 0x7fd6...,<function WildCardStrategy at 0x7fd6804696a8>,7,8,136,64
